In [1]:
import numpy as np
import pandas as pd
import torch.cuda

from src.myscripts import func
import config

# Data folder
data_path = config.SOUNDS_DATA_DIR_PATH

C:\Users\Bcom_\Documents\Projekty\Rozpoznawanie_dzwiekow_gitarowych\data\prepared_data


In [2]:
n_fft = 2048 # Ile próbek bierze do okna na ktorym dokonuje transformaty
hop_lenght = 1024 # O ile próbek przesuwa okno po każdej transformacie (Od tego zalezy wielkosc dataframe'a)
sr = 22050 # Liczba próbek na sekunde (Od tego zalezy wielkosc dataframe'a)

df = func.load_dataset(data_path, sr)
mel, chroma, contrast, labels = func.process_dataset(df, n_fft, hop_lenght)

mel = pd.DataFrame(np.array([row.flatten() for row in mel]))
chroma = pd.DataFrame(np.array([row.flatten() for row in chroma]))
contrast = pd.DataFrame(np.array([row.flatten() for row in contrast]))


In [3]:
mel.shape

(1451, 5632)

In [4]:
chroma.shape

(1451, 528)

In [5]:
contrast.shape

(1451, 308)

### Preparation of different dataframe versions

In [6]:
# Splitting labels

split_labels = map(lambda x: x.split("_", 1), labels)
sound_labels, combined_3_labels = zip(*split_labels)

sound_labels = list(sound_labels)
combined_3_labels = list(combined_3_labels)

string_labels, pluck_labels, sound_type_labels = zip(*map(lambda label: (label[0], label[1], label[2]), combined_3_labels))
string_labels, pluck_labels, sound_type_labels = list(string_labels), list(pluck_labels), list(sound_type_labels)
print(labels[0],":",sound_labels[0], string_labels[0], pluck_labels[0], sound_type_labels[0])

A2_spn : A2 s p n


Recognistions:
- sound recognition - mel_spectogram, chroma features
- string recognition - mel_spectogram, spectral contrast
- pluck recognition - mel_spectogram, spectral contrast
- sound_type recognition - mel_spectogram, spectral contrast

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
mel_normalized = scaler.fit_transform(mel)
chroma_normalized = scaler.fit_transform(chroma)
contrast_normalized = scaler.fit_transform(contrast)


In [8]:
sound_data = np.hstack((mel_normalized, chroma_normalized))
sound_data = pd.DataFrame(np.array([d.flatten() for d in sound_data]))
sound_data['label'] = sound_labels
print(sound_data.shape)

(1451, 6161)


In [9]:
string_data = np.hstack((mel_normalized, contrast_normalized))
string_data = pd.DataFrame(np.array([d.flatten() for d in string_data]))
string_data['label'] = string_labels
print(string_data.shape)

(1451, 5941)


In [10]:
pluck_data = np.hstack((mel_normalized, contrast_normalized))
pluck_data = pd.DataFrame(np.array([d.flatten() for d in pluck_data]))
pluck_data['label'] = pluck_labels
print(pluck_data.shape)

(1451, 5941)


In [11]:
sound_type_data = np.hstack((mel_normalized, contrast_normalized))
sound_type_data = pd.DataFrame(np.array([d.flatten() for d in sound_type_data]))
sound_type_data['label'] = sound_type_labels
print(sound_type_data.shape)

(1451, 5941)


In [12]:
# Dataset for predicting all classes at once.
data_combined = np.hstack((mel_normalized, chroma_normalized, contrast_normalized))
data_combined = pd.DataFrame(np.array([d.flatten() for d in data_combined]))
data_combined['label'] = labels
print(data_combined.shape)

(1451, 6469)


In [13]:
#Zapis do csv danych
sound_data.to_csv("../data/dataframes/sound_data.csv", index=False)
string_data.to_csv("../data/dataframes/string_data.csv", index=False)
pluck_data.to_csv("../data/dataframes/pluck_data.csv", index=False)
sound_type_data.to_csv("../data/dataframes/sound_type_data.csv", index=False)
data_combined.to_csv("../data/dataframes/data_combined.csv", index=False)

OSError: Cannot save file into a non-existent directory: '..\data\dataframes'